# Code for the Model

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout

IMG_SIZE = 224

datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.2
)

train_datagen = datagen.flow_from_directory(
    r"E:\New Plant Diseases Dataset\New Plant Diseases Dataset\train",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=16,
    class_mode='categorical',
    subset='training'
)

val_datagen = datagen.flow_from_directory(
    r"E:\New Plant Diseases Dataset\New Plant Diseases Dataset\valid",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=16,
    class_mode='categorical',
    subset='validation'
)

num_classes = 38

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

checkpoint = ModelCheckpoint(
    r"E:\New Plant Diseases Dataset\plant-224.h5",
    monitor='val_loss',
    mode='min',
    save_best_only=True,
    verbose=1
)

earlystop = EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=10,
    verbose=1,
    restore_best_weights=True
)

callbacks = [checkpoint, earlystop]

# Train the model and save the history
num_epochs = 5  # Change the number of epochs here
model_history = model.fit(
    train_datagen,
    validation_data=val_datagen,
    epochs=num_epochs,
    steps_per_epoch=train_datagen.samples // 16,
    validation_steps=val_datagen.samples // 16,
    callbacks=callbacks
)

# Plot the training and validation accuracy
acc_train = model_history.history['accuracy']
acc_val = model_history.history['val_accuracy']
epochs = range(1, len(acc_train) + 1)

plt.plot(epochs, acc_train, 'g', label='Training Accuracy')
plt.plot(epochs, acc_val, 'b', label='Validation Accuracy')
plt.title("Training and Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

# Losses

In [ ]:
# Collect loss data for both training and validation
loss_train = model_history.history['loss']
loss_val = model_history.history['val_loss']

# Check the minimum length of both arrays
min_length = min(len(loss_train), len(loss_val))

# Use the minimum length to adjust the number of epochs
epochs = range(1, min_length + 1)

plt.plot(epochs, loss_train[:min_length], 'g', label='Training Loss')
plt.plot(epochs, loss_val[:min_length], 'b', label='Validation Loss')
plt.title("Training and Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

# Confusion Matrix

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

IMG_SIZE = 224

datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.2
)

val_datagen = datagen.flow_from_directory(
    r"C:\Users\aksha\OneDrive\Desktop\COLLEGE\LW_Summer\plant proj\plant proj\New Plant Diseases Dataset\valid",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=16,
    class_mode='categorical',
    subset='validation'
)

# Load the pre-trained model from the h5 file
model_path = r"C:\Users\aksha\OneDrive\Desktop\COLLEGE\LW_Summer\plant proj\plant proj\models\plant-224.h5"
model = tf.keras.models.load_model(model_path)

# Make predictions on the validation data
val_predictions = model.predict(val_datagen)

# Convert the probabilities to class labels (indices of the maximum probability)
val_pred_labels = np.argmax(val_predictions, axis=1)

# Get the true class labels
val_true_labels = val_datagen.classes

conf_matrix = confusion_matrix(val_true_labels, val_pred_labels)

# Create a confusion matrix display
cm_display = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=val_datagen.class_indices.keys())

# Plot the confusion matrix
cm_display.plot(cmap='viridis', values_format='d')
plt.title("Confusion Matrix")
plt.show()